<a href="https://colab.research.google.com/github/RizkyFahrizal/capstone-project-C242-PS434/blob/main/ml/basedonDesc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sastrawi

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.7/209.7 kB 5.0 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import re
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
data = pd.read_csv('/content/sample_data/dataset_gabungan_clean.csv')
print(data.info)

<bound method DataFrame.info of                            Nama  \
0                        lappet   
1             kue adee meureudu   
2                       kelicuk   
3                       gonjeng   
4                   buah hongas   
..                          ...   
278                   cao menek   
279      kopi arabika kintamani   
280  kopi arabika flores bajawa   
281        kopi robusta lampung   
282          kopi liberika riau   

                                             Deskripsi  \
0    Lappet ini diolah dari beras yang disajikan de...   
1    Kue tradisional ini dibuat dari singkong yang ...   
2    Kudapan ini merupakan makanan pembuka sebelum ...   
3    Makanan ini bisa ditemui di pasar-pasar yang a...   
4    Dari namanya seperti buah-buahan, namun sebena...   
..                                                 ...   
278  minuman yang berasal dari suku Osing, terbuat ...   
279  Jenis kopi di Indonesia yang satu ini memiliki...   
280  Jenis kopi yang satu

In [ ]:
def preprocess(text):
    text = text.lower()
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    words = text.split()

    stemmer = StemmerFactory().create_stemmer()
    words = [stemmer.stem(word) for word in words]
    return words

data['tokenized_deskripsi'] = data['Deskripsi'].apply(preprocess)

In [ ]:
model_word2vec = Word2Vec(sentences=data['tokenized_deskripsi'] , vector_size=100, window=5, min_count=1, workers=4)

In [ ]:
def compute_sentence_embedding(tokenized_text, model):
    embeddings = []
    for word in tokenized_text:
        if word in model.wv:
            embeddings.append(model.wv[word])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(model.vector_size)

data['embedding'] = data['tokenized_deskripsi'].apply(lambda x: compute_sentence_embedding(x, model_word2vec))

In [ ]:
def rekomendasi(input_text, top_n=5):
    input_tokens = preprocess(input_text)
    input_embedding = compute_sentence_embedding(input_tokens, model_word2vec)

    similarity_scores = []
    for idx, row in data.iterrows():
        similarity = cosine_similarity([input_embedding], [row['embedding']])[0][0]
        similarity_scores.append((row['Nama'], similarity))

    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    recommended_ids = [x[0] for x in similarity_scores[:top_n]]
    return data[data['Nama'].isin(recommended_ids)]

In [ ]:
rekomendasi("makanan singkong", top_n=5)

,Nama,Deskripsi,Bahan,"Tags (makanan, minuman, camilan)","Tags (manis, gurih, asin, pedas, pahit)",Kalori (100g),Gula(g),Lemak(g),Karbohidrat(g),Serat(g),Protein(g),Sodium(mg),Vitamin C(mg),Vitamin A(IU),Asal,tokenized_deskripsi,embedding
2,kelicuk,Kudapan ini merupakan makanan pembuka sebelum ...,"pisang, beras ketan putih, kelapa parut, gula ...",camilan,"manis, gurih",120,8,3.0,22,1.0,2.0,50,1,30,"Palembang, Sumatera Selatan","[kudap, ini, rupa, makan, buka, belum, makan, ...","[-0.008117862, 0.013959924, 0.007548789, 0.000..."
46,bika bakar mandailing,Kabupaten Mandailing Natal memiliki makanan en...,"tepung singkong, tepung beras, gula pasir, san...",camilan,"manis, gurih",200,16,8.0,38,2.0,4.5,90,0,50,"Mandailing Natal, Sumatera Utara","[kabupaten, mandailing, natal, milik, makan, e...","[-0.00879406, 0.016095398, 0.008062418, -5.278..."
75,tili aya,Makanan ringan terbuat dari ketan yang dikukus...,"Ketan, kelapa",camilan,gurih,200,14,8.0,36,3.0,4.0,85,0,45,"Gorontalo, Gorontalo","[makan, ringan, buat, dari, ketan, yang, kukus...","[-0.015554718, 0.023990624, 0.0094213, 0.00167..."
133,tiwul,"Makanan yang terbuat dari singkong kering, bia...","Singkong, lauk pauk",makanan,"asin, gurih",270,6,12.0,35,3.0,7.0,350,8,60,"Yogyakarta, Yogyakarta","[makan, yang, buat, dari, singkong, kering, bi...","[-0.011926782, 0.023895644, 0.008045859, 0.002..."
134,gatot,Makanan khas Yogyakarta terbuat dari singkong ...,"Singkong, sambal",makanan,"asin, gurih",200,5,9.0,22,3.0,7.0,250,10,60,"Yogyakarta, Yogyakarta","[makan, khas, yogyakarta, buat, dari, singkong...","[-0.01255136, 0.024470111, 0.00671332, 0.00230..."
